# JuliaのTips集
このページはJuliaでの実装におけるTips (詰まったところの解決策)をまとめたものである．体系的にまとまってはいない．

## 関数名の!記号
単なる**慣習**として関数への入力を変更する場合に!を付ける．

関数内で配列を変更する場合には注意が必要である．以下に入力された配列を同じサイズの要素1の配列で置き換える，ということを目的として書かれた2つの関数がある．違いは`v`の後に`[:]`としているかどうかである．

In [1]:
function wrong!(a::Array)
    a = ones(size(a))
end

function right!(a::Array)
    a[:] = ones(size(a))
end

right! (generic function with 1 method)

実行すると`wrong!`の場合には入力された配列が変更されていないことがわかる．

In [2]:
using Random
v = rand(2, 2)
println("v : ", v)

wrong!(v)
println("wrong : ", v)

right!(v)
println("right : ", v)

v : [0.6280091741622842 0.9725327921727986; 0.5963608481273952 0.6912407844680865]
wrong : [0.6280091741622842 0.9725327921727986; 0.5963608481273952 0.6912407844680865]
right : [1.0 1.0; 1.0 1.0]


## 配列の1次元化
配列を一次元化(flatten)する方法．まずは3次元配列を作成する．

In [3]:
B = rand(2, 2, 2)

2×2×2 Array{Float64, 3}:
[:, :, 1] =
 0.949975   0.274306
 0.0413543  0.843118

[:, :, 2] =
 0.584134  0.465772
 0.947309  0.350395

用意されている`flatten`を素直に用いると次のようになる．

In [4]:
import Base.Iterators: flatten
collect(flatten(B))

8-element Vector{Float64}:
 0.9499751499139076
 0.04135434427279017
 0.27430559651250597
 0.8431176058769795
 0.5841338340589125
 0.9473085492415769
 0.4657724731039251
 0.35039483907714675

ただし，単に`B[:]`とするだけでもよい．

In [5]:
B[:]

8-element Vector{Float64}:
 0.9499751499139076
 0.04135434427279017
 0.27430559651250597
 0.8431176058769795
 0.5841338340589125
 0.9473085492415769
 0.4657724731039251
 0.35039483907714675

## 行列の行・列ごとの正規化
シミュレーションにおいてニューロン間の重み行列を行あるいは列ごとに正規化 (weight normalization)する場合がある．これは各ニューロンへの入力の大きさを同じにする働きや重みの発散を防ぐ役割がある．以下では行ごとの和を1にする．

In [6]:
W = rand(3,3)

3×3 Matrix{Float64}:
 0.672357  0.414045  0.315854
 0.997443  0.647668  0.909181
 0.713347  0.476189  0.345471

In [7]:
Wnormed = W ./ sum(W, dims=1)

3×3 Matrix{Float64}:
 0.28213  0.269227  0.201116
 0.41854  0.421138  0.57891
 0.29933  0.309636  0.219974

In [8]:
println(sum(Wnormed, dims=1))

[1.0 1.0 0.9999999999999999]


## 行列の結合 (concatenate)
行列の結合はMATLABに近い形式で行うことができる．まず，2つの行列A, Bを用意する．

In [9]:
A = [1 2; 3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

In [10]:
B = [4 5 6; 7 8 9]

2×3 Matrix{Int64}:
 4  5  6
 7  8  9

### 水平結合 (Horizontal concatenation)
`hcat`を使うやり方と，`[ ]`を使うやり方がある．

In [11]:
H1 = hcat(A,B)

2×5 Matrix{Int64}:
 1  2  4  5  6
 3  4  7  8  9

In [12]:
H2 = [A B]

2×5 Matrix{Int64}:
 1  2  4  5  6
 3  4  7  8  9

なお，MATLABのように次のようにすると正しく結合はされない．

In [13]:
H3 = [A, B]

2-element Vector{Matrix{Int64}}:
 [1 2; 3 4]
 [4 5 6; 7 8 9]

### 垂直結合 (Vertical concatenation)

In [14]:
V1 = vcat(A, B')

5×2 Matrix{Int64}:
 1  2
 3  4
 4  7
 5  8
 6  9

In [15]:
V2 = [A; B']

5×2 Matrix{Int64}:
 1  2
 3  4
 4  7
 5  8
 6  9

In [16]:
[V2 [A;B']]

5×4 Matrix{Int64}:
 1  2  1  2
 3  4  3  4
 4  7  4  7
 5  8  5  8
 6  9  6  9

## 配列に新しい軸を追加
要はnumpyでの`A[None, :]`や`A[np.newaxis, :]`のようなことがしたい場合．やや面倒だが，`reshape`を使うか，`[CartesianIndex()]`を用いる．

In [17]:
v = rand(3)

3-element Vector{Float64}:
 0.8350949038389617
 0.8440585152690242
 0.6286596434476082

In [18]:
newaxis = [CartesianIndex()]
v1 = v[newaxis, :]

1×3 Matrix{Float64}:
 0.835095  0.844059  0.62866

## Array{Array{Float64, x},1}をArray{Float64, x+1}に変換
numpyでは`array([matrix for i in range()])`などを用いると，1次元配列のリストを2次元配列に変換できた．Juliaでも同様にする場合は`hcat(...)`や`cat(...)`を用いる．

In [19]:
A1 = [i*rand(3) for i=1:5]

println("Type : ", typeof(A1))
println("Size : ", size(A1))

Type : Vector{Vector{Float64}}
Size : (5,)


In [20]:
A2 = hcat(A1...)'

println("Type : ", typeof(A2))
println("Size : ", size(A2))

Type : LinearAlgebra.Adjoint{Float64, Matrix{Float64}}
Size : (5, 3)


以下は多次元配列の場合．`cat(...)`で配列を結合し，`permitedims`で転置する．

In [21]:
B1 = [i*rand(3, 4, 5) for i=1:6]

println("Type : ", typeof(B1))
println("Size : ", size(B1))

Type : Vector{Array{Float64, 3}}
Size : (6,)


In [22]:
B2 = permutedims(cat(B1..., dims=4), [4, 1, 2, 3])

println("Type : ", typeof(B2))
println("Size : ", size(B2))

Type : Array{Float64, 4}
Size : (6, 3, 4, 5)


## 二項分布 (bernoulli distribution)のサンプリング

In [23]:
p = 0.7
N = 100

100

In [24]:
using BenchmarkTools

In [25]:
@benchmark floor.(p .+ rand(N))

BenchmarkTools.Trial: 10000 samples with 54 evaluations.
 Range (min … max):  862.963 ns … 62.243 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     901.852 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.068 μs ±  1.164 μs  ┊ GC (mean ± σ):  3.38% ± 4.35%

  █▆▄▃▃▂▁          ▂▁                                          ▁
  ███████▇█▆▆▇▇▆▆▆▇███▇▇▆▆▆▆▅▅▅▅▅▃▅▄▅▅▃▄▅▄▃▃▂▃▂▃▄▄▂▄▂▃▄▃▄▃▃▂▂▂ █
  863 ns        Histogram: log(frequency) by time      3.33 μs <

 Memory estimate: 1.91 KiB, allocs estimate: 7.

In [26]:
println(sum(floor.(p .+ rand(N)) .== 1.0) / N) 

0.67


In [27]:
@benchmark 1.0f0 * (p .≥ rand(N))

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  970.000 ns … 147.660 μs  ┊ GC (min … max): 0.00% … 98.34%
 Time  (median):       1.100 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.345 μs ±   2.661 μs  ┊ GC (mean ± σ):  3.18% ±  1.71%

   █▄▁                                                           
  ▄███▇▄▃▂▂▃▃▃▃▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▂▂▂▂ ▃
  970 ns           Histogram: frequency by time         3.35 μs <

 Memory estimate: 1.53 KiB, allocs estimate: 6.

In [28]:
println(sum(1.0f0 * (p .≥ rand(N)) .== 1.0) / N) 

0.66


## Roth's column lemma

Roth's column lemmaは，例えば，$A, B, C$が与えられていて，$X$を未知とするときの方程式 $AXB = C$を考えると，この方程式は

$$
(B^\top \otimes A)\text{vec}(X) = \text{vec}(AXB)=\text{vec}(C)
$$

の形に書き下すことができる，というものである．$\text{vec}(\cdot)$はvec作用素（行列を列ベクトル化する作用素）である．`vec(X) = vcat(X...)`で実現できる．Roth's column lemmaを用いれば，$AXB = C$の解は

$$
X = \text{vec}^{-1}\left((B^\top \otimes A)^{-1}\text{vec}(C)\right)
$$

として得られる．ただし，$\text{vec}(\cdot)^{-1}$は列ベクトルを行列に戻す作用素(inverse of the vectorization operator)である．`reshape()`で実現できる．2つの作用素をまとめると，

$$
\begin{align}
\text{vec} &: R^{m\times n}\to R^{mn}\\
\text{vec}^{−1} &: R^{mn}\to R^{m×n}
\end{align}
$$

であり，$\text{vec}^{−1}\left(\text{vec}(X)\right)=X\ (\text{for all}\ X\in R^{m\times n})，\text{vec}\left(\text{vec}^{−1}(x)\right)=x\ (\text{for all}\ x \in R^{mn})$となる．

In [29]:
using LinearAlgebra, Kronecker, Random

In [30]:
m = 4
A = randn(m, m)
B = randn(m, m)
C = convert(Array{Float64}, reshape(1:16, (m, m)))

4×4 Matrix{Float64}:
 1.0  5.0   9.0  13.0
 2.0  6.0  10.0  14.0
 3.0  7.0  11.0  15.0
 4.0  8.0  12.0  16.0

In [31]:
vec(X) = vcat(X...)

vec (generic function with 1 method)

In [32]:
X = reshape((B' ⊗ A) \ vec(C), (m, m))

4×4 Matrix{Float64}:
 14.9263   -32.6353   -1.59895   6.15749
 -7.68572   -1.32912   9.22409   1.95084
  7.8402    -3.73689  -7.05015  -0.55172
 -8.2306    -8.55244  13.1808    4.10262

In [33]:
A * X * B

4×4 Matrix{Float64}:
 1.0  5.0   9.0  13.0
 2.0  6.0  10.0  14.0
 3.0  7.0  11.0  15.0
 4.0  8.0  12.0  16.0

## "splat" operator ... 
https://docs.julialang.org/en/v1/base/base/#...

## broadcastingの回避

In [34]:
foo(a,b) = sum(a) + b

foo (generic function with 1 method)

In [35]:
println(foo.(Ref([1,2]),[3,4,5]))
println(foo.(([1,2],), [3,4,5]))
println(foo.([[1,2]], [3,4,5]))

[6, 7, 8]
[6, 7, 8]
[6, 7, 8]


## `var`の使用

In [36]:
var"log(θ+μ)" = 10

10

## reshapeにおける残りの次元の指定
numpyにおいては(2, 3, 5)次元の配列に対し，reshape(-1, 5)を行うと(6, 5)次元の配列となった．これと同様なことは，Juliaでは:を使うことで実装できる．

In [37]:
a = rand(2,3,5)
b = reshape(a, (:, 5))

6×5 Matrix{Float64}:
 0.618538  0.589642  0.505904   0.882143  0.802949
 0.020081  0.327577  0.414112   0.922128  0.0954408
 0.324497  0.345642  0.0254123  0.759426  0.928881
 0.748477  0.585216  0.95366    0.89351   0.717935
 0.515622  0.398863  0.92236    0.834462  0.190159
 0.221536  0.276032  0.687437   0.430937  0.315746